In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Stable Diffusion Inpainting

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_inpainting.ipynb">
      <img alt="Workbench logo" src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" width="32px"><br> Run in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_pytorch_stable_diffusion_inpainting.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_diffusion_inpainting.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates deploying [runwayml/stable-diffusion-inpainting](https://huggingface.co/runwayml/stable-diffusion-inpainting) on Vertex AI for Online Prediction.

### Objective

- Upload the model to [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).
- Deploy the model to a [Vertex AI Endpoint resource](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints).
- Run online predictions for image-inpainting.

### File a bug

File a bug on [GitHub](https://github.com/GoogleCloudPlatform/vertex-ai-samples/issues/new) if you encounter any issue with the notebook.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Run the notebook

In [ ]:
# @title Setup Google Cloud project

# Import the necessary packages

# Upgrade Vertex AI SDK.
! pip3 install --upgrade --quiet 'google-cloud-aiplatform==1.103.0'

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

BUCKET_URI = "gs://"  # @param {type:"string"}

# @markdown 3. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

import importlib
import os
from io import BytesIO

import requests
from google.cloud import aiplatform
from PIL import Image

if os.environ.get("VERTEX_PRODUCT") != "COLAB_ENTERPRISE":
    ! pip install --upgrade tensorflow
! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION)

! gcloud config set project $PROJECT_ID

aiplatform.init(project=PROJECT_ID, location=REGION)

models, endpoints = {}, {}
LABEL = "endpoint"

import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
)

# @markdown Set use_dedicated_endpoint to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint). Note that [dedicated endpoint does not support VPC Service Controls](https://cloud.google.com/vertex-ai/docs/predictions/choose-endpoint-type), uncheck the box if you are using VPC-SC.
use_dedicated_endpoint = True  # @param {type:"boolean"}

In [ ]:
# @title Deploy

# @markdown This section uploads the stable diffusion model for the `image-inpainting` task using one L4 GPU.
# @markdown This step will take ~15 minutes to complete.

# @markdown Select one of the two OSS models:
# @markdown * [kandinsky-community/kandinsky-2-2-decoder-inpaint](https://huggingface.co/kandinsky-community/kandinsky-2-2-decoder-inpaint)
# @markdown * [diffusers/stable-diffusion-xl-1.0-inpainting-0.1](https://huggingface.co/diffusers/stable-diffusion-xl-1.0-inpainting-0.1)

model_id = "kandinsky-community/kandinsky-2-2-decoder-inpaint"  # @param ["kandinsky-community/kandinsky-2-2-decoder-inpaint", "diffusers/stable-diffusion-xl-1.0-inpainting-0.1"] {isTemplate:true}

# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve-opt:20240605_1400_RC00"

accelerator_type = "NVIDIA_L4"
machine_type = "g2-standard-8"
accelerator_count = 1

task = "image-inpainting"


def deploy_model(
    model_id: str,
    task: str,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
):
    model_name = model_id
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
        "DEPLOY_SOURCE": "notebook",
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        model_garden_source_model_name="publishers/runwayml/models/stable-diffusion-inpainting",
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="g2-standard-8",
        accelerator_type="NVIDIA_L4",
        accelerator_count=1,
        deploy_request_timeout=1800,
        system_labels={
            "NOTEBOOK_NAME": "model_garden_pytorch_stable_diffusion_inpainting.ipynb",
            "NOTEBOOK_ENVIRONMENT": common_util.get_deploy_source(),
        },
    )
    return model, endpoint


models[LABEL], endpoints[LABEL] = deploy_model(
    model_id=model_id,
    task=task,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)

In [ ]:
# @title Predict

# @markdown Once deployment succeeds, you can send text prompt and image to the endpoint to generated images.

# @markdown When deployed on one L4 GPU, the average inference time of a request is ~2.5 seconds.

# @markdown You may adjust the parameters below to achieve best image quality.

# @markdown Note: The image uri can be an http uri or gcs uri (starting with "gs://"). You may upload your own images to gcs bucket and provide the gcs uri.

prompt = "a black cat with glowing eyes, cute, adorable, disney, pixar, highly detailed"  # @param {type: "string"}
negative_prompt = ""  # @param {type: "string"}
init_image_uri = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png"  # @param {type: "string"}
mask_image_uri = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png"  # @param {type: "string"}
num_inference_steps = 25  # @param {type:"number"}

image_dimension = 512
if model_id == "diffusers/stable-diffusion-xl-1.0-inpainting-0.1":
    image_dimension = 1024


def download_image(url):
    if url.startswith("gs://"):
        storage_client = storage.Client()
        blob = storage_client.bucket(url.split("/")[2]).get_blob(url.split("/", 3)[-1])
        return Image.open(BytesIO(blob.download_as_bytes())).convert("RGB")
    else:
        response = requests.get(url)
        return Image.open(BytesIO(response.content)).convert("RGB")


init_image = download_image(init_image_uri).resize((image_dimension, image_dimension))
mask_image = download_image(mask_image_uri).resize((image_dimension, image_dimension))

instances = [
    {
        "prompt": prompt,
        "image": common_util.image_to_base64(init_image),
        "mask_image": common_util.image_to_base64(mask_image),
        "num_inference_steps": num_inference_steps,
        "negative_prompt": negative_prompt,
    },
]
response = endpoints[LABEL].predict(instances=instances)
images = [common_util.base64_to_image(image) for image in response.predictions]
display(common_util.image_grid([init_image, mask_image, images[0]], rows=1, cols=3))

In [ ]:
# @title Clean up resources

# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()